In [ ]:
#필요한 모듈들을 import
import numpy as np 
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt

# 파일을 불러올 때 사용하기 위해서 address에 cwd를 저장한다.
address = os.getcwd()

# writeFile() 함수 정의 (파일을 생성할 때 사용) ----------------------------------------------------------------------------------------
#가계부 항목들을 columns으로 가진 dataframe을 생성하여 csv파일로 저장한다.
def writeFile(): 
        
    print('\n-----------------------------------------------------------------------------\nPlease enter a date. ex) 2018 01 05') 
    year=str(input('YEAR(XXXX) : '))
    month = str(input('MONTH(YY) : '))
    day = str(input('DAY(ZZ) : '))
    today = (year+' - '+month+' - '+day)
    pre_balance=int(input('Enter the balance you currently have.: '))
    first_row = [today,'.','.','.','.',pre_balance]
    new_frameFile = pd.DataFrame([first_row],columns = ['date','sort','category','detail','amount','balance'])
    new_frameFile.to_csv(user_name+'\'s_File.csv', index=False)
    
    # 동시에 사용자가 saving하는 자산들을 기록할 파일을 따로 생성하여 역시 csv파일로 저장한다.
    # saving파일의 기조잔액은 0원이다.
    first_row_saving = [today,'.',',',',',0]
    new_file_saving = pd.DataFrame([first_row_saving],columns=['date','category','detail','amount','balance'])
    new_file_saving.to_csv(user_name+'\'s_File_saving.csv', index=False)
    
    new_frameFile
    print(user_name+"'s File created.")

#----------------------------------------------------------------------------------------------------------------------------------------



# record()함수 정의(사용자가 내역 입력을 사용할 경우 실행)------------------------------------------------------------------------------
def record():
    global data
    new_frameFile = pd.read_csv(address+'\\'+user_name+'\'s_File.csv', engine = 'python', index_col=False)

    #날짜 형식이 통일되도록 > 년, 월, 일 각각 숫자로 입력받아 통일된 형식으로 저장
    print('\n-----------------------------------------------------------------------------\nPlease enter a date. ex) 2018 01 05')
    year=str(input('YEAR(XXXX) : '))
    month = str(input('MONTH(YY) : '))
    day = str(input('DAY(ZZ) : '))
    today = (year+' - '+month+' - '+day)

    # 데이터에 입력될 것들을 '항목_set'의 변수 이름으로 저장한다.
    # sort의 종류에 따라 내역기입의 실행방식이 달라져야 한다.
    # 따라서 sort_in에 숫자로 선택하도록 한다. 
    print('[1. Income  2.Spending  3.Saving ]')
    sort_in = int(input('Please enter types of usage.(choose the number) : '))
    # sort_in이 무엇인지에 따라서 category에 입력할 내용이 달라진다. 각 경우별 category에 입력할 내용의 예시들을 print한다.
    if sort_in == 1:
        print('\nex) Pocket money, Salary, Financial income, Business profits, Rental income, etc\n')
    if sort_in == 2:
        print('\nex) Food, Transportation fee, Clothing, Cultural expense, Necessaries, Beauty, Educational expense,\n   Medical expense, Housing costs, Communication expenses, Interest, Tax, etc\n')
    if sort_in == 3:
        print('Please enter your account to save.\n( ex)KB Bank )\n ')
    
    # 나머지 항목들을 마저 입력받는다.
    category_set = input('Please enter the category of the input. : ')
    detail_set = input('Enter the details. : ')
    amount_set = int(input('Enter the amount. : '))        
    #balance cell 기존 데이터와 합쳐진 후 계산된 값이 저장될 것이므로, 일단 여기서는 blank string 저장 
    balance_set = ''
    
    # 위에서 사용자가 선택한 sort_in에 해당하는 것을 str으로 입력되도록 저장.
    if sort_in == 1:
        sort_set = ('Income')
    elif sort_in == 2:
        amount_set = -(amount_set)
        sort_set = ('Spending')
    elif sort_in == 3:
        amount_set = -(amount_set)
        sort_set = ('Saving')
        # 사용자가 saving을 한 경우에는 전에 만들어둔 saving 용 파일에도 내역을 따로 저장해야한다.
        # 밑에 내역 기록하는 것과같은 과정을 saving 파일 내에서도 동일하게 실행한다.(과정은 밑과 같으므로 주석 생략)
        new_file_saving = pd.read_csv(address+'\\'+user_name+'\'s_File_saving.csv', engine = 'python', index_col=False)
        
        new_row_saving = [today,category_set,detail_set,amount_set,balance_set]
        record_saving = pd.DataFrame([new_row_saving],columns = ['date','category','detail','amount','balance'])
        
        combi_saving = new_file_saving.append(record_saving, ignore_index = True)
        
        combi_saving['amount'][len(combi_saving)-1]= - (combi_saving['amount'][len(combi_saving)-1])
        balance_cal_saving = int(combi_saving['balance'][len(combi_saving)-2])+int(combi_saving['amount'][len(combi_saving)-1])
        combi_saving['balance'][len(combi_saving)-1] = balance_cal_saving
        
        combi_saving.to_csv(user_name+'\'s_File_saving.csv',index=False)
        
        
    #기존 데이터에 새로 기록될 row의 내용들을 새로운 dataframe(record_File이라는 기록용 df)에 저장한다.
    new_row = [today,sort_set,category_set,detail_set,amount_set,balance_set]
    record_File = pd.DataFrame([new_row],columns = ['date','sort','category','detail','amount','balance'])
    
    #기존 data에 새로운 dataframe을 append하여 결과적으로, 새로운 내역을 기록할 때마다 row가 추가되는 형식으로 만든다.
    combi = new_frameFile.append(record_File, ignore_index = True)
    
    #내역이 추가된 후 잔액을 계산한다
    balance_cal = int(combi['balance'][len(combi)-2])+int(combi['amount'][len(combi)-1])
    combi['balance'][len(combi)-1] = balance_cal
    
    
    #잔액 계산까지 마친 최종 파일을 기존 파일과 동일한 이름으로 저장한다(덮어쓴다).
    combi.to_csv(user_name+'\'s_File.csv', index=False)
    data = pd.read_csv(user_name+'\'s_File.csv')
    
    
    # 경고메시지 띄우기
    # income,spending,saving 금액을 합할 변수들을 미리 지정한다.
    # saving도 결국 사용가능자산에서는 빠져나가는 것이므로 withdrawal에 지출과 같이 합한다.
    sum_usable = float(data['balance'][0])
    sum_withdrawal = 0
        
    #for문을 이용하여 데이터의 처음부터 끝까지 각 sort별 합을 구해 위에서 설정한 변수들에 저장한다. 
    for a in range(1,len(data)):
        if data['sort'][a] == 'Income':
            #sum_usable 은 사용가능자산들로 시작잔액에 sort가 income인 row들의 합을 더한다.
            sum_usable = float(sum_usable) + float(data['amount'][a])
        elif data['sort'][a] == 'Spending' or 'Saving':
            sum_withdrawal = float(sum_withdrawal) + float(data['amount'][a])

    # 내역의 지출항목의 합이 수입항목의 50%를 초과하였으면 경고메시지를 출력한다.
    if -(sum_withdrawal) > (sum_usable*(0.5)) :
        print('\n\n****************※ Warning ※*****************\n')
        print('Your expenses exceeded 50% of available assets.\nYou should cut back on your expenses.\n')
        print('**********************************************\n')

    # 잔액 기입을 마친 후의 내역을 프린트하여 보여준다.
    print('=============================================================================')
    print(data)

#----------------------------------------------------------------------------------------------------------------------------------------


# correcting() 함수 정의 (사용자가 수정을 사용할 경우 실행)-------------------------------------------------------------------------------
def correcting():
    global data
    
    # 수정하기 위해 기존 파일을 읽어온다.
    cort_data = pd.read_csv(user_name+'\'s_File.csv')
    # 사용자가 무엇을 수정할지 선택해야 하므로, 기존 내역을 print해준다.
    print(cort_data)
    
    # 수정할 내역의 index와 무슨 항목을 수정할지 선택받는다.
    cort_row = int(input('Choose the index number you want to correct. : '))
    cort_col = input('Choose the content you want to correct. : \n( date, sort, category, detail, amount, balance )')
    # 사용자가 date항목을 수정할 경우 통일된 날짜 형식으로 저장되도록 한다.
    if cort_col == 'date':
        print('\n-----------------------------------------------------------------------------\nPlease enter a new date. ex) 2018 01 05') 
        year=str(input('YEAR(XXXX) : '))
        month = str(input('MONTH(YY) : '))
        day = str(input('DAY(ZZ) : '))
        cort_data[cort_col][cort_row] = (year+' - '+month+' - '+day)
    else:
        # 나머지 항목들은 어떻게 수정하고 싶은지 물어보고 입력받는다.
        cort_data[cort_col][cort_row]=input( 'How do you want to change? ')
    
    # 사용자가 amount나 balance를 수정할 경우 잔액을 재계산한다.
    if cort_col == 'amount' or 'balance':
        if cort_row == 0:
            for i in range(cort_row+1,len(cort_data)):
                cort_data['balance'][i]=int(cort_data['balance'][i-1])+int(cort_data['amount'][i])
        else:
            for i in range(cort_row,len(cort_data)):
                cort_data['balance'][i]=int(cort_data['balance'][i-1])+int(cort_data['amount'][i])
    
    # 수정을 마친 파일을 기존 파일과 동일한 이름으로 저장한다.(덮어쓴다)    
    cort_data.to_csv(user_name+'\'s_File.csv', index=False)
    data = pd.read_csv(user_name+'\'s_File.csv')
    
    # 수정을 완료한 내역을 출력하여 보여준다.
    print('=============================================================================')
    print(data)
    print('=============================================================================\n\nComplete.')
    

#----------------------------------------------------------------------------------------------------------------------------------------        
    


# remove()함수 정의(사용자가 내역 삭제를 선택할 경우 사용)-------------------------------------------------------------------------------
def remove():
    # 내역 삭제를 위해서 del_data라는 변수로 기존 파일을 불러온다.
    del_data = pd.read_csv(user_name+'\'s_File.csv')
    print(del_data)
    
    # 삭제할 내역을 입력받아 del_row에 저장한 후, .drop method를 이용하여 삭제
    del_row = int(input('Choose the index number you want to delete: '))
    del_data = del_data.drop(index = del_row)
    
    # 중간 index가 빠져버렸으므로 index를 재정렬한다.
    del_data.index = pd.RangeIndex(len(del_data.index))
    del_data.index = range(len(del_data.index))
    
    # 중간 내역이 빠져 삭제된 내역부터 잔액이 맞지 않으므로, 잔액을 재계산한다.
    for i in range(1,len(del_data)):
        del_data['balance'][i]=int(del_data['balance'][i-1])+int(del_data['amount'][i])

    # 삭제가 완료된 파일을 data에 저장하고, 삭제 완료된 내역을 print하여 보여준다.
    del_data.to_csv(user_name+'\'s_File.csv', index=False)
    data = pd.read_csv(user_name+'\'s_File.csv')
    
    print('=============================================================================')
    print(data)
    print('=============================================================================\n\nComplete.')
#----------------------------------------------------------------------------------------------------------------------------------------------



# analysis() 함수 정의(사용자가 분석 기능을 선택할 경우 사용)---------------------------------------------------------------------------------
def analysis():
    data = pd.read_csv(user_name+'\'s_File.csv')
    
    sum_income = 0
    sum_spending = 0
    sum_saving = 0
       
    #for문을 이용하여 데이터의 처음부터 끝까지 각 sort별 합을 구해 위에서 설정한 변수들에 저장한다. 
    for a in range(1,len(data)):
        if data['sort'][a] == 'Income':
            #sum_usable 은 사용가능자산들로 시작잔액에 sort가 income인 row들의 합을 더한다.
            sum_income = float(sum_income) + float(data['amount'][a])
        elif data['sort'][a] == 'Spending':
            sum_spending = float(sum_spending) + float(data['amount'][a])
        elif data['sort'][a] == 'Saving':
            sum_saving = float(sum_saving)+float(data['amount'][a])
    
    # spending과 saving은 -로 입력되고 있으므로 통계를 위해서 부호를 바꿔서 저장한다. 
    sum_spending = -(sum_spending)
    sum_saving = -(sum_saving)
    
    sum_total = (sum_income)+(sum_spending)+(sum_saving)
    
    # 각 항목의 자치 비율을 구한다.
    ratio_income = (sum_income)/(sum_total)
    ratio_spending = (sum_spending)/(sum_total)
    ratio_saving = (sum_saving)/(sum_total)
    
    # 각 항목의 차지 비율을 print하여 보여준다.
    print('+=======================< Total >========================+\n')
    print('                  income : '+str(sum_income)+' won \n')
    print('                spending : '+str(sum_spending)+' won \n')
    print('                  saving : '+str(sum_saving)+' won \n')
    #그래프의 색, lable, 비율을 설정하고 pie차트로 프린트한다.
    colors = ['lightskyblue','lightpink','gold']
    labels = ['Income','Spending','Saving']
    ratio = [ratio_income, ratio_spending, ratio_saving]
    plt.pie(ratio, labels=labels,colors=colors, shadow=True, startangle=90)
    plt.show()
    
#----------------------------------------------------------------------------------------------------------------------------------------       


   
print('**************************** Welcome ***********************************\n')

user_name = input('Please let me know your name. : ')
#파일을 생성할 것인가 불러올 것인가 사용자가 정하도록 한다.

#사용자가 가계부 생성을 선택하면, writeFile() 함수를 실행
while True:
    mode_set = int(input('Choose the option.\n\n1. Create data       2. Continue yours\n\n'))
    if mode_set == 1:
        # 만약 사용자가 이미 파일이 있는데 1을 선택하였을 경우, 새로 생성하면 이전 파일이 삭제되므로 경고메시지를 띄운다.
        if os.path.exists(address+'\\'+user_name+'\'s_File.csv')== True:
            print('---------------------------------< Warning >----------------------------------------\n')
            print('Your file already exists. If you continue to this, The existing file will be deleted.\nDo you still want to continue? \n')
            print('------------------------------------------------------------------------------------')
            # 그래도 실행할 것인지 물어보고 대답이 yes면 새로 생성(이전파일지워짐)
            answer = int(input('1. Yes  2. No (Please choose the number.)'))
            if answer == 1:
                writeFile()
                print('   ** Done **\n Enter < 2. Continue yours > to continue running this program.')
            # 대답이 No면 option선택창으로 돌아간다.
            if answer == 2:
                print('\nYou chose \'No\'.>> Return to option selection!')
        else:
            writeFile()
            print('   ** Done **\n Enter < 2. Continue yours > to continue running this program.')
             
    
    # 이어서쓰기를 선택하면 미리 쓰던 가계부에서 무슨 작업을 할 것인지 선택하는 text를 우선 프린트한다.
    elif mode_set == 2:
        while True:
            #만약 사용자가 이어서쓰기를 선택하였으나, 기존 파일이 없는 경우 writeFile() 함수를 실행하여 파일을 생성하고 시작한다.
            if os.path.exists(address+'\\'+user_name+'\'s_File.csv')== False:
                print('\n  ** Nope **\nYour file does not exist .\n\n>> Create your File.\n')
                writeFile()
            else:
                break
        using = 0    
        while using==0:
            #기능 선택 메뉴
            print('+------------------------------+')
            print('       (1) Write       ')
            print('       (2) Correcting       ')
            print('       (3) Remove      ')
            print('       (4) Analysis        ')
            print('+------------------------------+')
        
            #메뉴를 선택할 것을 번호로 입력하도록 input받는다. 그 외의 것을 선택하면 프로그램을 종료하는 것을 선택하는 것으로 간주
            print('\nChoose the Number you want to use. ')
            print('If you want to EXIT, choose the other one. : ')
            choice = int(input())

            #사용자가 작업을 선택하면 각 작업에 해당하는 함수를 실행한다.         
            if choice == 1:
                record()
            elif choice == 2:
                correcting()
            elif choice ==3:
                remove()
            elif choice == 4:
                analysis()
            else:
                # 사용자가 종료를 선택하면(1,2,3,4) 외 다른 것 진짜 프로그램을 나갈 것인지 묻는다.
                print('\nAre you sure you want to exit the program?')

            # 진짜 프로그램을 나갈 것인지 물은 후와
            # 가계부에서 작업을 마친 후엔, 프로그램을 계속할 것인지 물어본다. 0을 입력할 경우 기능 선택 메뉴가 재실행된다.
            using=int(input('\nEnter 0 to continue using the program, or the other to Exit.\n\n'))
        # 정말 나가기를 선택하면 프로그램이 종료된다.
        if using != 0:
            break
            

    else:
        print('Please choose 1 or 2.')#맨 처음 파일 선택 옵션에서 1이나 2를 선택하지 않았을 경우, 옵션 창으로 되돌아간다.
        

# 현재까지 한 작업 파일을 data에 읽기로 저장하며 파일 종료시 dataframe형태로 print하여 보여준다.
data = pd.read_csv(user_name+'\'s_File.csv',engine='python')

print('\n~~~~~~~~~~~~~~~~~~~ <'+user_name+'\'s Household Accounts > ~~~~~~~~~~~~~~~~~~~~~')
data

